Đọc dữ liệu, loại bỏ các trường dữ liệu không cần thiết

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'data_proccessed/chungcu_preprocess.csv'
df = pd.read_csv(file_path)

# Drop columns not needed for prediction
df = df.drop(columns=['Link', 'Giá tổng', 'Hướng cửa chính', 'Hướng ban công'])

# Separate target and features
X = df.drop(columns=['Giá/m²'])
y = df['Giá/m²']

Tính toán hệ số tương quan của trường Giá/m² với các trường dữ liệu còn lại

In [32]:
from sklearn.preprocessing import LabelEncoder

# Chọn các cột có kiểu dữ liệu string
categorical_columns = df.select_dtypes(include=['object']).columns

# Chuyển các cột chuỗi thành số bằng LabelEncoder
label_encoder = LabelEncoder()

for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col].astype(str))  # Chuyển đổi cột chuỗi thành số

# Tính ma trận tương quan giữa các cột
correlation_matrix = df.corr()

# Lấy hệ số tương quan của "Giá/m2" với các cột khác
price_per_sqm_correlation = correlation_matrix["Giá/m²"].drop(labels=["Giá/m²"], errors='ignore')

# In ra hệ số tương quan
print(price_per_sqm_correlation)

Số phòng ngủ           0.723724
Số phòng vệ sinh       0.676813
Giấy tờ pháp lý        0.090372
Loại hình căn hộ       0.017365
Diện tích             -0.002419
Tình trạng nội thất    0.019961
Xã/Phường             -0.057459
Quận/Huyện            -0.018028
Name: Giá/m², dtype: float64


Độ tương quan thấp -> cần sử dụng các mô hình phi tuyến để đạt hiệu quả cao hơn.

In [17]:
y

0        50.46
1        33.00
2        42.06
3        33.82
4        41.67
         ...  
2258    104.49
2259     72.30
2260     56.00
2261     85.53
2262     88.66
Name: Giá/m², Length: 2263, dtype: float64

Mã hóa dữ liệu, áp dụng mô hình 

In [ ]:
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Preprocessing for categorical data: impute missing values and apply one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for numerical data: impute missing values with mean
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the model
model = RandomForestRegressor(random_state=0)

# Create the full pipeline with preprocessing and model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f" Root Mean Squared Error: {rmse}")
rmse

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")


Mean Squared Error: 279.9711359708676
 Root Mean Squared Error: 16.732338030618063
Mean Absolute Error: 9.101031383029353
R2 Score: 0.819627660755881


c:\Users\This Pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
pip install xgboost

In [36]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

# Đọc dữ liệu
df = pd.read_csv('data_proccessed/chungcu_preprocess.csv')

# Loại bỏ các cột không cần thiết
df = df.drop(columns=['Link', 'Giá tổng', 'Hướng cửa chính', 'Hướng ban công'])

# One-Hot Encoding cho các cột hạng mục
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)

# Tách dữ liệu thành đầu vào (X) và đầu ra (y)
X = df.drop(columns=['Giá/m²'])
y = df['Giá/m²']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Khởi tạo mô hình XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Khởi tạo mô hình XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f" Root Mean Squared Error: {rmse}")
rmse

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

Mean Squared Error: 275.3181755212573
 Root Mean Squared Error: 16.592714531421834
Mean Absolute Error: 9.861935642206525
R2 Score: 0.8226253460629618


c:\Users\This Pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
